# Predicción de despacho de productos
## Alternativa 1: Modelo por cada producto

In [3]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm
from skopt.space import Integer, Categorical


In [4]:
pivot_df = pd.read_csv('../data/pivot_df_interpolado.csv')

In [5]:
pivot_df.shape

(262805, 38)

In [6]:
# Drop rows where all columns except 'product_id' and 'customer_id' are null
cols_to_check = pivot_df.columns.difference(['product_id', 'customer_id'])
pivot_df = pivot_df.dropna(subset=cols_to_check, how='all')
pivot_df.head()

,product_id,customer_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,20001,10001,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,20001,10002,35.72806,6.79415,29.94128,22.81133,31.22847,47.57025,21.84874,17.08052,...,31.97079,55.41679,30.87299,144.07021,37.14616,0.00000,72.08551,17.40806,45.61495,113.33165
2,20001,10003,143.49426,20.48319,137.87537,68.89292,135.12190,171.01785,64.66196,83.63410,...,170.89924,230.00152,1.84835,0.00000,138.23391,162.07198,233.20532,76.00625,86.14415,102.27517
3,20001,10004,184.72927,104.03894,295.43924,247.65632,188.37819,195.02683,379.44270,237.16848,...,102.64484,91.67799,389.02653,66.71971,228.62366,96.11402,288.34205,324.96172,195.67828,34.64810
4,20001,10005,19.08407,5.17117,5.17117,0.86186,37.95546,19.08407,43.35049,67.53856,...,6.90049,22.18016,15.89578,0.00000,8.25595,0.00000,12.80400,17.13921,12.22149,19.60368


In [7]:
pivot_df.shape

(262805, 38)

In [8]:
df_201912_sum = pivot_df.groupby('product_id', as_index=False)['201912'].sum()
df_201912_sum = df_201912_sum.rename(columns={'201912': 'tn'})
df_201912_sum.head()

,product_id,tn
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443


In [9]:
df_201912_sum.to_csv('suma_201912.csv', index=False)

In [10]:
# Calcula el promedio de los últimos 3 periodos (meses) para cada producto
last_3_periods = cols_to_check[-3:]  # Selecciona los nombres de las últimas 3 columnas de periodo
df_last3_avg = pivot_df.groupby('product_id', as_index=False)[last_3_periods].sum()
df_last3_avg['avg_last3'] = df_last3_avg[last_3_periods].mean(axis=1)
df_last3_avg = df_last3_avg[['product_id', 'avg_last3']]
df_last3_avg.to_csv('promedio_ultimos_3.csv', index=False)

In [11]:
# Calcula el promedio de los últimos 6 periodos (meses) para cada producto
last_6_periods = cols_to_check[-6:]  # Selecciona los nombres de las últimas 6 columnas de periodo
df_last6_avg = pivot_df.groupby('product_id', as_index=False)[last_6_periods].sum()
df_last6_avg['avg_last6'] = df_last6_avg[last_6_periods].mean(axis=1)
df_last6_avg = df_last6_avg[['product_id', 'avg_last6']]
df_last6_avg.to_csv('promedio_ultimos_6.csv', index=False)

In [12]:
# Calcula el promedio de los últimos 9 periodos (meses) para cada producto
last_9_periods = cols_to_check[-9:]  # Selecciona los nombres de las últimas 9 columnas de periodo
df_last9_avg = pivot_df.groupby('product_id', as_index=False)[last_9_periods].sum()
df_last9_avg['avg_last9'] = df_last9_avg[last_9_periods].mean(axis=1)
df_last9_avg = df_last9_avg[['product_id', 'avg_last9']]
df_last9_avg.to_csv('promedio_ultimos_9.csv', index=False)

In [13]:
# Calcula el promedio de los últimos 12 periodos (meses) para cada producto
last_12_periods = cols_to_check[-12:]  # Selecciona los nombres de las últimas 12 columnas de periodo
df_last12_avg = pivot_df.groupby('product_id', as_index=False)[last_12_periods].sum()
df_last12_avg['avg_last12'] = df_last12_avg[last_12_periods].mean(axis=1)
df_last12_avg = df_last12_avg[['product_id', 'avg_last12']]
df_last12_avg.to_csv('promedio_ultimos_12.csv', index=False)

In [14]:
# Calcula el promedio de los últimos 24 periodos (meses) para cada producto
last_24_periods = cols_to_check[-24:]  # Selecciona los nombres de las últimas 24 columnas de periodo
df_last24_avg = pivot_df.groupby('product_id', as_index=False)[last_24_periods].sum()
df_last24_avg['avg_last24'] = df_last24_avg[last_24_periods].mean(axis=1)
df_last24_avg = df_last24_avg[['product_id', 'avg_last24']]
df_last24_avg.to_csv('promedio_ultimos_24.csv', index=False)

In [15]:
# Extrapola el dato de febrero 2020 usando regresión lineal sobre febreros de 2017, 2018 y 2019 para cada producto

feb_years = ['201702', '201802', '201902']
target_feb = 202002

extrapolaciones = []

for prod_id in pivot_df['product_id'].unique():
    # Suma por producto para cada febrero
    vals = pivot_df[pivot_df['product_id'] == prod_id][feb_years].sum()
    if vals.isnull().any():
        continue  # Si falta algún dato, salta este producto
    X = np.array([201702, 201802, 201902]).reshape(-1, 1)
    y = vals.values
    model = LinearRegression()
    model.fit(X, y)
    pred_202002 = model.predict(np.array([[target_feb]]))[0]
    extrapolaciones.append({'product_id': prod_id, 'feb_2020_pred': pred_202002})

df_feb_2020_pred = pd.DataFrame(extrapolaciones)
df_feb_2020_pred.head()

,product_id,feb_2020_pred
0,20001,1494.702273
1,20002,1290.760723
2,20003,772.461123
3,20004,418.021887
4,20005,311.981860


In [16]:
# Calcula la variación porcentual de febrero respecto a diciembre anterior para cada año y producto
variaciones = []

for prod_id in pivot_df['product_id'].unique():
    # Suma por producto para cada diciembre y febrero siguiente
    dic_2017 = pivot_df[pivot_df['product_id'] == prod_id]['201712'].sum()
    feb_2018 = pivot_df[pivot_df['product_id'] == prod_id]['201802'].sum()
    dic_2018 = pivot_df[pivot_df['product_id'] == prod_id]['201812'].sum()
    feb_2019 = pivot_df[pivot_df['product_id'] == prod_id]['201902'].sum()
    dic_2019 = pivot_df[pivot_df['product_id'] == prod_id]['201912'].sum()
    
    # Evita división por cero y NaN
    if dic_2017 > 0 and dic_2018 > 0 and dic_2019 > 0:
        var_2018 = (feb_2018 - dic_2017) / dic_2017
        var_2019 = (feb_2019 - dic_2018) / dic_2018
        # Promedio de variaciones
        coef = np.mean([var_2018, var_2019])
        # Predicción para feb 2020
        pred_feb_2020 = dic_2019 * (1 + coef)
        variaciones.append({'product_id': prod_id, 'coef': coef, 'pred_feb_2020': pred_feb_2020})
    else:
        variaciones.append({'product_id': prod_id, 'coef': np.nan, 'pred_feb_2020': np.nan})

df_feb_2020_varpct = pd.DataFrame(variaciones)
df_feb_2020_varpct.head()

,product_id,coef,pred_feb_2020
0,20001,-0.079223,1385.482125
1,20002,-0.049543,1033.440137
2,20003,-0.097604,805.389639
3,20004,-0.180104,523.011523
4,20005,0.155913,685.738777


In [17]:
df_feb_2020_pred[['product_id', 'feb_2020_pred']].rename(columns={'feb_2020_pred': 'tn'}).to_csv('prediccion_feb_2020_simple.csv', index=False)

In [18]:
# Proyecta el valor de febrero 2020 para cada producto usando regresión lineal sobre todos los periodos disponibles

proyecciones = []

# Obtén los nombres de las columnas de periodo (excluyendo product_id y customer_id)
period_cols = cols_to_check

for prod_id in pivot_df['product_id'].unique():
    # Suma por producto para cada periodo
    vals = pivot_df[pivot_df['product_id'] == prod_id][period_cols].sum()
    # Elimina NaN
    vals = vals.dropna()
    if len(vals) < 3:
        continue  # Necesitamos al menos 3 puntos para ajustar una recta
    X = np.array(vals.index.astype(int)).reshape(-1, 1)
    y = vals.values
    model = LinearRegression()
    model.fit(X, y)
    pred_202002 = model.predict(np.array([[202002]]))[0]
    proyecciones.append({'product_id': prod_id, 'feb_2020_proj': pred_202002})

df_feb_2020_proj = pd.DataFrame(proyecciones)
df_feb_2020_proj.head()

,product_id,feb_2020_proj
0,20001,1625.306476
1,20002,1420.131566
2,20003,684.969974
3,20004,580.565549
4,20005,634.207010


In [19]:
df_feb_2020_proj.rename(columns={'feb_2020_proj': 'tn'}).to_csv('prediccion_feb_2020_proj.csv', index=False)

In [20]:
# Calcula el MAPE (Mean Absolute Percentage Error) entre y y el valor real de 201912
# y es el array de valores reales, model es el modelo LinearRegression ya entrenado

# Predicción para todos los periodos
y_pred = model.predict(X)

# Evita división por cero
mask = y != 0
mape = np.mean(np.abs((y[mask] - y_pred[mask]) / y[mask])) * 100 if np.any(mask) else np.nan

print(f"MAPE: {mape:.2f}%")

MAPE: 208.45%


In [21]:
from sklearn.ensemble import RandomForestRegressor

predicciones_sum = []

for prod_id in pivot_df['product_id'].unique():
    serie_sum = pivot_df[pivot_df['product_id'] == prod_id][cols_to_check].sum()
    serie_sum = serie_sum.dropna()
    if len(serie_sum) < 8 or '201912' not in serie_sum.index:
        continue

    X_sum = np.array(serie_sum.index.astype(int)).reshape(-1, 1)
    y_sum = serie_sum.values

    train_mask_sum = X_sum.flatten() <= 201910
    test_mask_sum = X_sum.flatten() == 201912

    X_train_sum, y_train_sum = X_sum[train_mask_sum], y_sum[train_mask_sum]
    X_test_sum, y_test_sum = X_sum[test_mask_sum], y_sum[test_mask_sum]

    if len(X_train_sum) < 8 or len(y_test_sum) == 0:
        continue

    model_sum = RandomForestRegressor(
        random_state=42, n_estimators=250, max_depth=5,
        min_samples_split=8, min_samples_leaf=1, max_features='sqrt', bootstrap=True
    )
    model_sum.fit(X_train_sum, y_train_sum)
    pred_201912_sum = model_sum.predict(np.array([[201912]]))[0]

    predicciones_sum.append({
        'product_id': prod_id,
        'pred_201912_sum': pred_201912_sum,
        'real_201912_sum': y_test_sum[0]
    })

df_predicciones_sum = pd.DataFrame(predicciones_sum)
df_predicciones_sum.head()

,product_id,pred_201912_sum,real_201912_sum
0,20001,1506.421814,1504.68856
1,20002,1580.735236,1087.30855
2,20003,849.979216,892.50129
3,20004,874.809398,637.90002
4,20005,850.349466,593.24443


In [22]:
from sklearn.ensemble import RandomForestRegressor

predicciones_feb2020 = []

for prod_id in pivot_df['product_id'].unique():
    serie_sum = pivot_df[pivot_df['product_id'] == prod_id][cols_to_check].sum()
    serie_sum = serie_sum.dropna()
    if len(serie_sum) < 8 or '201912' not in serie_sum.index:
        continue

    X_sum = np.array(serie_sum.index.astype(int)).reshape(-1, 1)
    y_sum = serie_sum.values

    # Entrenamos con todos los datos disponibles
    X_train_sum, y_train_sum = X_sum, y_sum

    # Predicción para febrero 2020 (202002)
    X_pred = np.array([[202002]])

    model_sum = RandomForestRegressor(
        random_state=42, n_estimators=250, max_depth=5,
        min_samples_split=8, min_samples_leaf=1, max_features='sqrt', bootstrap=True
    )
    model_sum.fit(X_train_sum, y_train_sum)
    pred_202002_sum = model_sum.predict(X_pred)[0]

    predicciones_feb2020.append({
        'product_id': prod_id,
        'pred_202002_sum': pred_202002_sum
    })

df_predicciones_feb2020 = pd.DataFrame(predicciones_feb2020)
df_predicciones_feb2020.head()


,product_id,pred_202002_sum
0,20001,1488.839995
1,20002,1446.227233
2,20003,917.726407
3,20004,785.338614
4,20005,767.283339


In [23]:
df_predicciones_feb2020.rename(columns={'pred_202002_sum': 'tn'}).to_csv('prediccion_feb_2020_rf.csv', index=False)

In [24]:
df_merged = pd.merge(
    df_201912_sum,
    df_predicciones_feb2020,
    on='product_id',
    how='inner'
)
df_merged.head()


,product_id,tn,pred_202002_sum
0,20001,1504.68856,1488.839995
1,20002,1087.30855,1446.227233
2,20003,892.50129,917.726407
3,20004,637.90002,785.338614
4,20005,593.24443,767.283339


In [25]:
df_merged['avg_tn_pred_202002_sum'] = (df_merged['tn'] + df_merged['pred_202002_sum']) / 2
df_merged.head()

,product_id,tn,pred_202002_sum,avg_tn_pred_202002_sum
0,20001,1504.68856,1488.839995,1496.764277
1,20002,1087.30855,1446.227233,1266.767891
2,20003,892.50129,917.726407,905.113849
3,20004,637.90002,785.338614,711.619317
4,20005,593.24443,767.283339,680.263885


In [26]:
df_merged = df_merged.drop(columns=['tn', 'pred_202002_sum'])
df_merged.head()

,product_id,avg_tn_pred_202002_sum
0,20001,1496.764277
1,20002,1266.767891
2,20003,905.113849
3,20004,711.619317
4,20005,680.263885


In [27]:
df_merged.rename(columns={'avg_tn_pred_202002_sum': 'tn'}).to_csv('promedio_prediccion_feb_2020.csv', index=False)

In [28]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

predicciones_feb2020 = []

for prod_id in pivot_df['product_id'].unique():
    serie_sum = pivot_df[pivot_df['product_id'] == prod_id][cols_to_check].sum()
    serie_sum = serie_sum.dropna()
    if len(serie_sum) < 8 or '201912' not in serie_sum.index:
        continue

    # Convertir el índice a fechas para SARIMAX
    serie_sum.index = pd.to_datetime(serie_sum.index.astype(str), format='%Y%m')
    serie_sum = serie_sum.sort_index()

    try:
        # Entrenar SARIMAX con todos los datos menos el último
        model = SARIMAX(serie_sum.iloc[:-1], order=(1,1,1), seasonal_order=(1,1,1,4))
        model_fit = model.fit(disp=False)
        # Predicción para febrero 2020
        pred_202002 = model_fit.predict(start=pd.to_datetime('2020-02-01'), end=pd.to_datetime('2020-02-01')).iloc[0]
        predicciones_feb2020.append({
            'product_id': prod_id,
            'pred_202002_sum': pred_202002
        })
    except Exception:
        continue

df_predicciones_feb2020_sarimax = pd.DataFrame(predicciones_feb2020)
df_predicciones_feb2020_sarimax.head()


/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(da

,product_id,pred_202002_sum
0,20001,1529.190705
1,20002,1328.610699
2,20003,879.694429
3,20004,775.025998
4,20005,695.267540


In [29]:
df_predicciones_feb2020_sarimax.rename(columns={'pred_202002_sum': 'tn'}).to_csv('prediccion_feb_2020_sarimax.csv', index=False)

In [31]:
df_last12_avg.head()

,product_id,avg_last12
0,20001,1454.732720
1,20002,1175.437142
2,20003,784.976407
3,20004,627.215328
4,20005,668.270104


In [34]:
# Multiplica los valores de 'avg_last12' en df_last12_avg por un valor aleatorio (media=1, std=0.1)
np.random.seed(42)  # Para reproducibilidad
random_factors = np.random.normal(loc=1, scale=0.1, size=len(df_last12_avg))
df_multiplicado = df_last12_avg.copy()
df_multiplicado['avg_last12'] = df_multiplicado['avg_last12'] * random_factors
df_multiplicado.head()



,product_id,avg_last12
0,20001,1526.991353
1,20002,1159.185042
2,20003,835.818430
3,20004,722.742095
4,20005,652.622334


In [35]:
df_multiplicado.rename(columns={'avg_last12': 'tn'}).to_csv('prediccion_feb_2020_avg_last12_factor01.csv', index=False)

In [36]:
from statsmodels.tsa.arima.model import ARIMA

# Crear un DataFrame con la suma por producto para cada periodo
df_sum_by_product = pivot_df.groupby('product_id')[cols_to_check].sum().reset_index()

predicciones_feb2020 = []

for idx, row in df_sum_by_product.iterrows():
    prod_id = row['product_id']
    serie_sum = row[cols_to_check]
    serie_sum = serie_sum.dropna()
    if len(serie_sum) < 8 or '201912' not in serie_sum.index:
        continue

    # Convertir el índice a fechas para ARIMA
    serie_sum.index = pd.to_datetime(serie_sum.index.astype(str), format='%Y%m')
    serie_sum = serie_sum.sort_index()

    try:
        # Entrenar ARIMA con todos los datos menos el último
        model = ARIMA(serie_sum.iloc[:-1], order=(1,1,1))
        model_fit = model.fit()
        # Predicción para febrero 2020
        pred_202002 = model_fit.predict(start=pd.to_datetime('2020-02-01'), end=pd.to_datetime('2020-02-01')).iloc[0]
        predicciones_feb2020.append({
            'product_id': prod_id,
            'tn': pred_202002
        })
    except Exception:
        continue

df_predicciones_feb2020_arima = pd.DataFrame(predicciones_feb2020)
df_predicciones_feb2020_arima.head()


/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/media/n

,product_id,tn
0,20001.0,1483.422724
1,20002.0,1325.512849
2,20003.0,977.371354
3,20004.0,626.528213
4,20005.0,631.439614


In [37]:
df_predicciones_feb2020_arima.to_csv('prediccion_feb_2020_arima.csv', index=False)

In [49]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
cnt = 0
predicciones_feb2020 = []

for prod_id in pivot_df['product_id'].unique():
    serie_sum = pivot_df[pivot_df['product_id'] == prod_id][cols_to_check].sum()
    serie_sum = serie_sum.dropna()
    # Convertir el índice a fechas para SARIMAX
    serie_sum.index = pd.to_datetime(serie_sum.index.astype(str), format='%Y%m')
    serie_sum = serie_sum.sort_index()
    if len(serie_sum) < 8:
        # Si hay menos de 8 datos, usar el promedio de los últimos 12 meses
        avg_last12 = serie_sum[-12:].mean() if len(serie_sum) > 0 else np.nan
        predicciones_feb2020.append({
            'product_id': prod_id,
            'tn': avg_last12
        })
        cnt += 1
    else:
        # Si hay suficientes datos, entrenar SARIMAX
        model = SARIMAX(serie_sum.iloc[:-1], order=(1,1,1), seasonal_order=(1,1,1,12))
        model_fit = model.fit(disp=False)
        # Predicción para febrero 2020
        pred_202002 = model_fit.predict(start=pd.to_datetime('2020-02-01'), end=pd.to_datetime('2020-02-01')).iloc[0]
        predicciones_feb2020.append({
            'product_id': prod_id,
            'tn': pred_202002
        })


df_predicciones_feb2020_sarimax_ajustado = pd.DataFrame(predicciones_feb2020)
df_predicciones_feb2020_sarimax_ajustado.head()


/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization 

,product_id,tn
0,20001,1057.272204
1,20002,1154.452166
2,20003,523.499164
3,20004,365.714188
4,20005,619.106387


In [50]:
print(cnt)

0


In [51]:
df_predicciones_feb2020_sarimax_ajustado.to_csv('prediccion_feb_2020_sarimax_ajustado_12_meses.csv', index=False)

In [ ]:
pivot_df = pd.read_csv('../data/pivot_df_interpolado.csv')

In [58]:
import pmdarima as pm

cnt = 0
predicciones_feb2020 = []

for prod_id in pivot_df['product_id'].unique():
    serie_sum = pivot_df[pivot_df['product_id'] == prod_id][cols_to_check].sum()
    serie_sum = serie_sum.dropna()
    # Convertir el índice a fechas para auto_arima
    serie_sum.index = pd.to_datetime(serie_sum.index.astype(str), format='%Y%m')
    serie_sum = serie_sum.sort_index()
    if len(serie_sum) < 8:
        # Si hay menos de 8 datos, usar el promedio de los últimos 12 meses
        avg_last12 = serie_sum[-12:].mean() if len(serie_sum) > 0 else np.nan
        predicciones_feb2020.append({
            'product_id': prod_id,
            'tn': avg_last12
        })
        cnt += 1
    else:
        # Ajuste con pmdarima.auto_arima
        try:
            model = pm.auto_arima(serie_sum.iloc[:-1], seasonal=True, m=12, suppress_warnings=True, error_action='ignore')
            pred_202002 = model.predict(n_periods=1)[0]
        except Exception:
            pred_202002 = np.nan
        predicciones_feb2020.append({
            'product_id': prod_id,
            'tn': pred_202002
        })

df_predicciones_feb2020_autoarima = pd.DataFrame(predicciones_feb2020)
df_predicciones_feb2020_autoarima.head()

/tmp/ipykernel_238366/2818956451.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_202002 = model.predict(n_periods=1)[0]
/tmp/ipykernel_238366/2818956451.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_202002 = model.predict(n_periods=1)[0]
/tmp/ipykernel_238366/2818956451.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_202002 = model.predict(n_periods=1)[0]
/tmp/ipykernel_238366/2818956451.py:24: Future

,product_id,tn
0,20001,1484.284071
1,20002,1169.083615
2,20003,762.724605
3,20004,508.993943
4,20005,549.536116


In [59]:
df_predicciones_feb2020_autoarima.to_csv('prediccion_feb_2020_autoarima.csv', index=False)

In [16]:
# Normaliza los valores de febrero para cada producto a escala 0-1 y multiplica por el promedio de los últimos 12 meses

resultados = []

for prod_id in pivot_df['product_id'].unique():
    vals = pivot_df[pivot_df['product_id'] == prod_id][feb_years].sum()
    if vals.isnull().any():
        continue
    feb_min = vals.min()
    feb_max = vals.max()
    if feb_max == feb_min:
        escala_feb = 1.0  # Si todos los valores son iguales, asigna 1
    else:
        escala_feb = (vals[-1] - feb_min) / (feb_max - feb_min)  # Normaliza el último febrero (2019)
    # Predicción lineal para febrero 2020
    X = np.array([int(y) for y in feb_years]).reshape(-1, 1)
    y = vals.values
    model = LinearRegression()
    model.fit(X, y)
    pred_202002 = model.predict(np.array([[target_feb]]))[0]
    # Busca el promedio de los últimos 12 meses para este producto
    avg_last12 = df_last12_avg[df_last12_avg['product_id'] == prod_id]['avg_last12'].values
    avg_last12 = avg_last12[0] if len(avg_last12) > 0 else np.nan
    resultado = escala_feb * avg_last12
    resultados.append({'product_id': prod_id, 'escala_feb': escala_feb, 'avg_last12': avg_last12, 'resultado': resultado})

df_resultado_escala = pd.DataFrame(resultados)
df_resultado_escala.head()

/tmp/ipykernel_228898/3686180369.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  escala_feb = (vals[-1] - feb_min) / (feb_max - feb_min)  # Normaliza el último febrero (2019)
/tmp/ipykernel_228898/3686180369.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  escala_feb = (vals[-1] - feb_min) / (feb_max - feb_min)  # Normaliza el último febrero (2019)
/tmp/ipykernel_228898/3686180369.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.ilo

,product_id,escala_feb,avg_last12,resultado
0,20001,1.000000,1454.732720,1454.732720
1,20002,1.000000,1175.437142,1175.437142
2,20003,0.171621,784.976407,134.718718
3,20004,0.000000,627.215328,0.000000
4,20005,0.070231,668.270104,46.933407


In [17]:
df_resultado_escala[['product_id', 'resultado']].rename(columns={'resultado': 'tn'}).to_csv('resultado_escala.csv', index=False)

In [15]:
df_feb_2020_pred.to_csv('prediccion_feb_2020.csv', index=False)

In [ ]:
model = SARIMAX(serie.iloc[:-1], order=(1,1,1), seasonal_order=(1,1,1,4))

In [17]:
from itertools import product
from skopt import BayesSearchCV
from skopt.space import Categorical
from sklearn.base import BaseEstimator, RegressorMixin

i = 20001
# 20001.0	10005.0	
serie = pivot_df[(pivot_df['product_id'] == 20001) & (pivot_df['customer_id'] == 10001)].iloc[0, 2:]  # Exclude 'product_id' and 'customer_id' columns
# serie = pivot_df[(pivot_df['product_id'] == 20001) & (pivot_df['customer_id'] == 10005)].iloc[0, 2:]  # Exclude 'product_id' and 'customer_id' columns
serie = serie.dropna()

# Convert index to datetime for SARIMAX
# Assuming your index is in 'YYYYMM' format
serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')
serie = serie.sort_index()

# Define training and testing periods
train_end_date = '2019-10-31'
test_date = '2019-12-01' # For prediction

# Split data
train_data = serie[serie.index <= train_end_date]
test_data = serie[serie.index == test_date]
# Wrapper para SARIMAX compatible con sklearn
class SARIMAXWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, order=(1,1,1), seasonal_order=(1,1,0,4)):
        self.order = order
        self.seasonal_order = seasonal_order
        self.model_ = None
        self.model_fit_ = None

    def fit(self, X, y):
        self.model_ = SARIMAX(y, order=self.order, seasonal_order=self.seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        self.model_fit_ = self.model_.fit(disp=False)
        return self

    def predict(self, X):
        dates = pd.to_datetime(np.array(X).flatten())
        preds = []
        for date in dates:
            pred = self.model_fit_.predict(start=date, end=date)
            preds.append(pred.iloc[0])
        return np.array(preds)

# Espacio de búsqueda
param_space = {
    'order': Categorical([
        (0,0,0), (0,0,1), (0,1,0), (0,1,1), (0,2,1),
        (1,0,0), (1,0,1), (1,1,0), (1,1,1), (1,1,2), (1,2,1), (1,2,2),
        (2,0,0), (2,0,1), (2,0,2), (2,1,0), (2,1,1), (2,1,2), (2,2,1), (2,2,2),
        (3,0,0), (3,0,1), (3,1,0), (3,1,1), (3,1,2), (3,2,1), (3,2,2)
    ]),
    'seasonal_order': Categorical([
        (0,0,0,4), (0,1,0,4), (1,0,0,4), (1,1,0,4), (1,1,1,4), (2,1,0,4), (0,1,1,4), (1,0,1,4), (2,0,1,4),
        (1,2,0,4), (2,2,0,4), (2,2,1,4), (1,2,1,4), (2,1,1,4), (2,0,2,4), (1,0,2,4), (0,2,1,4), (0,2,0,4),
        (0,0,0,12), (0,1,0,12), (1,0,0,12), (1,1,0,12), (1,1,1,12), (2,1,0,12), (0,1,1,12), (1,0,1,12), (2,0,1,12),
        (1,2,0,12), (2,2,0,12), (2,2,1,12), (1,2,1,12), (2,1,1,12), (2,0,2,12), (1,0,2,12), (0,2,1,12), (0,2,0,12)
    ])
}

# Prepara los datos para sklearn (X: fechas, y: valores)
X_train_sklearn = train_data.index.values.reshape(-1, 1)
y_train_sklearn = train_data.values

opt = BayesSearchCV(
    SARIMAXWrapper(),
    param_space,
    n_iter=20,
    cv=[(np.arange(len(X_train_sklearn)), np.arange(len(X_train_sklearn)))],  # No split, solo fit
    scoring='neg_mean_squared_error',
    n_jobs=1,
    verbose=1
)

opt.fit(X_train_sklearn, y_train_sklearn)

# Mejor modelo
best_model = opt.best_estimator_
best_params = (best_model.order, best_model.seasonal_order)

# Predicción para diciembre 2019
pred_201912 = best_model.predict(np.array([pd.to_datetime(test_date)]))[0]

print(f"Mejor combinación: order={best_params[0]}, seasonal_order={best_params[1]}")
print(f"Predicción para product_id={i} en 2019-12: {pred_201912:.5f}")
if not test_data.empty:
    print(f"Valor real: {test_data.iloc[0]:.5f}")
else:
    print("No hay valor real disponible para comparación.")

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
i = 20001
# 20001.0	10005.0	
# serie = pivot_df_interpolado[(pivot_df_interpolado['product_id'] == 20001) & (pivot_df_interpolado['customer_id'] == 10001)].iloc[0, 2:]  # Exclude 'product_id' and 'customer_id' columns
serie = pivot_df_interpolado[(pivot_df_interpolado['product_id'] == 20001) & (pivot_df_interpolado['customer_id'] == 10005)].iloc[0, 2:]  # Exclude 'product_id' and 'customer_id' columns
serie = serie.dropna()

# Convert index to datetime for SARIMAX
# Assuming your index is in 'YYYYMM' format
serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')
serie = serie.sort_index()

# Define training and testing periods
train_end_date = '2019-10-31'
test_date = '2019-12-01' # For prediction

# Split data
train_data = serie[serie.index <= train_end_date]
test_data = serie[serie.index == test_date]



param_space = {
    'order': Categorical([
        (1,1,1), (2,1,1), (1,1,2), (2,1,2), (0,1,1), (1,0,1)
    ]),
    'seasonal_order': Categorical([
        (1,1,0,4), (1,1,1,4), (2,1,0,4), (0,1,1,4), (1,0,1,4),
        (1,1,0,12), (1,1,1,12), (2,1,0,12)
    ])
}



# Initialize and fit the SARIMAX model
# 'enforce_stationarity=False' and 'enforce_invertibility=False' can help with convergence
model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
model_fit = model.fit(disp=False) # disp=False to suppress optimization output

# Make prediction for December 2019
start_index = pd.to_datetime(test_date)
end_index = pd.to_datetime(test_date)

# Predict using the fitted model
pred_201912 = model_fit.predict(start=start_index, end=end_index)

print(f"Predicción para product_id={i} en 2019-12: {pred_201912.iloc[0]:.5f}")

if not test_data.empty:
    print(f"Valor real: {test_data.iloc[0]:.5f}")
    # Optional: Calculate and print RMSE for evaluation
    # rmse = np.sqrt(mean_squared_error(test_data, pred_201912))
    # print(f"RMSE: {rmse:.5f}")
else:
    print("No hay valor real disponible para comparación.")

In [ ]:
df_first_2500 = pivot_df_interpolado.head(2500).copy()
df_first_2500.head()

In [ ]:
order = (1,1,1)
seasonal_order = (2,1,0,4) 

In [ ]:
resultados = []

for idx, row in df_first_2500.iterrows():
    print(f"Processing index: {idx}")
    product_id = row['product_id']
    customer_id = row['customer_id']
    serie = row[2:]
    serie = serie.dropna()
    if len(serie) < 8 or '201912' not in serie.index.astype(str):
        continue

    # Convertir el índice a fechas
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')
    serie = serie.sort_index()

    # Definir periodos de train y test
    train_end_date = '2019-10-31'
    test_date = '2019-12-01'

    train_data = serie[serie.index <= train_end_date]
    test_data = serie[serie.index == test_date]

    if len(train_data) < 8 or test_data.empty:
        continue

    try:
        model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)
        pred_201912 = model_fit.predict(start=pd.to_datetime(test_date), end=pd.to_datetime(test_date)).iloc[0]
        real_201912 = test_data.iloc[0]
        resultados.append({
            'product_id': product_id,
            'customer_id': customer_id,
            'sarimax_pred_201912': pred_201912,
            'real_201912': real_201912
        })
    except Exception:

        continue

df_resultados = pd.DataFrame(resultados)
df_resultados.head()


In [ ]:
df_resultados.head(200)

In [ ]:
# Calcula el error porcentual absoluto medio (MAPE) en sarimax_pred_201912
mape_df_resultados = np.mean(np.abs((df_resultados['real_201912'] - df_resultados['sarimax_pred_201912']) / df_resultados['real_201912'])) * 100
print(f"Error porcentual absoluto medio (MAPE) en df_resultados: {mape_df_resultados:.2f}%")

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
from sklearn.base import BaseEstimator, RegressorMixin

# Wrapper para SARIMAX compatible con sklearn
class SARIMAXWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, order=(1,1,1), seasonal_order=(1,1,0,4)):
        self.order = order
        self.seasonal_order = seasonal_order
        self.model_ = None
        self.model_fit_ = None

    def fit(self, X, y):
        # SARIMAX espera un índice de fechas
        self.model_ = SARIMAX(y, order=self.order, seasonal_order=self.seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        self.model_fit_ = self.model_.fit(disp=False)
        return self

    def predict(self, X):
        # X can be a 2D array (n_samples, 1) or 1D array of timestamps
        # Convert to 1D array of pandas Timestamps
        dates = pd.to_datetime(np.array(X).flatten())
        preds = []
        for date in dates:
            pred = self.model_fit_.predict(start=date, end=date)
            preds.append(pred.iloc[0])
        return np.array(preds)

# Espacio de búsqueda
param_space = {
    'order': Categorical([(1,1,1), (2,1,1), (1,1,2)]),
    'seasonal_order': Categorical([(1,1,0,4), (1,1,1,4), (2,1,0,4)])
}

# Prepara los datos para sklearn (X: fechas, y: valores)
X_train_sklearn = train_data.index.values.reshape(-1, 1)
y_train_sklearn = train_data.values

opt = BayesSearchCV(
    SARIMAXWrapper(),
    param_space,
    n_iter=10,
    cv=[(np.arange(len(X_train_sklearn)), np.arange(len(X_train_sklearn)))],  # No split, solo fit
    scoring='neg_mean_squared_error',
    n_jobs=1,
    verbose=1
)

opt.fit(X_train_sklearn, y_train_sklearn)

# Mejor modelo
best_model = opt.best_estimator_

# Predicción para diciembre 2019
pred_201912 = best_model.predict(np.array([start_index]))[0]

print(f"Predicción optimizada para product_id={i} en 2019-12: {pred_201912:.5f}")
if not test_data.empty:
    print(f"Valor real: {test_data.iloc[0]:.5f}")
else:
    print("No hay valor real disponible para comparación.")

In [ ]:
sarimax_preds = []
sarimax_real = []
product_ids = []

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]
    serie = serie.dropna()
    if len(serie) < 8 or 201912 not in serie.index:
        continue  # Necesitamos suficientes datos y valor real en 201912

    # Convertir el índice a fechas para SARIMAX
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')

    # Entrenar hasta octubre 2019
    train_data = serie[serie.index <= '2019-10-31']
    test_data = serie[serie.index == '2019-12-01']

    try:
        model = SARIMAX(train_data, order=(1,1,1), seasonal_order=(1,1,0,4),
                        enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)
        pred = model_fit.predict(start=pd.to_datetime('2019-12-01'), end=pd.to_datetime('2019-12-01'))
        sarimax_preds.append(pred.iloc[0])
        sarimax_real.append(test_data.iloc[0])
        product_ids.append(prod_id)
    except Exception:
        continue

df_sarimax_pred = pd.DataFrame({
    'product_id': product_ids,
    'sarimax_pred_201912': sarimax_preds,
    'real_201912': sarimax_real
})

df_sarimax_pred.head()


In [ ]:
# Calcula el error porcentual absoluto medio (MAPE) entre las predicciones SARIMAX y los valores reales
mape_sarimax = np.mean(np.abs((df_sarimax_pred['real_201912'] - df_sarimax_pred['sarimax_pred_201912']) / df_sarimax_pred['real_201912'])) * 100
print(f"Error porcentual absoluto medio (MAPE) SARIMAX: {mape_sarimax:.2f}%")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


# Selecciona el producto i (por ejemplo, product_id = 20001)
i = 20003
serie = pivot_df[pivot_df['product_id'] == i].iloc[0, 1:]  # Excluye la columna product_id
serie = serie.dropna()

# Prepara los datos: X = mes (como entero), y = tn
X = np.array(serie.index.astype(int)).reshape(-1, 1)
y = serie.values

# Train: hasta octubre 2019 (201910)
train_mask = X.flatten() <= 201910
X_train, y_train = X[train_mask], y[train_mask]

# Test: diciembre 2019 (201912)
test_mask = X.flatten() == 201912
X_test, y_test = X[test_mask], y[test_mask]

# Modelo simple: regresión lineal sobre el tiempo
# model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
# model = RandomForestRegressor(random_state=42)
model = RandomForestRegressor(random_state=42, n_estimators=250, max_depth=5, min_samples_split=8, min_samples_leaf=1, max_features='sqrt', bootstrap=True)
model = RandomForestRegressor(random_state=42, n_estimators=250, max_depth=5, min_samples_split=8, min_samples_leaf=1, max_features='sqrt', bootstrap=True)

# model = LinearRegression()

model.fit(X_train, y_train)

# Predicción para diciembre 2019 (201912)
pred_201912 = model.predict(np.array([[201912]]))[0]

print(f"Predicción para product_id={i} en 2019-12: {pred_201912:.5f}")
if len(y_test) > 0:
    print(f"Valor real: {y_test[0]:.5f}")
else:
    print("No hay valor real disponible para comparación.")

In [ ]:
from skopt.space import Integer, Categorical
from skopt import BayesSearchCV
iter = 0

# Definir el espacio de búsqueda de hiperparámetros
param_space = {
    'n_estimators': Categorical([50, 100, 150, 200, 250, 300]),#Integer(50, 400),
    'max_depth': Categorical([1, 5, 10, 20]),
    'min_samples_split':  Categorical([2, 4, 6, 8, 10]),
    'min_samples_leaf':  Categorical([1, 3, 5, 7]),
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True])
}


# Optimización bayesiana con logging
opt = BayesSearchCV(
    RandomForestRegressor(random_state=42),
    param_space,
    n_iter=100,
    cv=3,
    n_jobs=1,
    random_state=42,
    scoring='neg_mean_squared_error',
    verbose=1
)

opt.fit(X_train, y_train)
model = opt.best_estimator_

# Predicción para diciembre 2019 (201912)
pred_201912 = model.predict(np.array([[201912]]))[0]

print(f"Predicción para product_id={i} en 2019-12: {pred_201912:.5f}")
if len(y_test) > 0:
    print(f"Valor real: {y_test[0]:.5f}")
else:
    print("No hay valor real disponible para comparación.")

In [ ]:
# Creamos un diccionario para almacenar las predicciones
predicciones = {}

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]  # Excluye la columna product_id
    serie = serie.dropna()
    if len(serie) < 2:
        continue  # No se puede ajustar un modelo con menos de 2 puntos

    X_prod = np.array(serie.index.astype(int)).reshape(-1, 1)
    y_prod = serie.values

    # Train: hasta octubre 2019 (201910)
    train_mask = X_prod.flatten() <= 201910
    X_train_prod, y_train_prod = X_prod[train_mask], y_prod[train_mask]

    # Modelo simple: regresión lineal sobre el tiempo
    if len(X_train_prod) < 2:
        continue  # No se puede ajustar un modelo con menos de 2 puntos

    # model_prod = LinearRegression()
    model_prod = RandomForestRegressor(random_state=42, n_estimators=250, max_depth=5, min_samples_split=8, min_samples_leaf=1, max_features='sqrt', bootstrap=True)
    model_prod.fit(X_train_prod, y_train_prod)

    # Predicción para diciembre 2019 (201912)
    pred_201912 = model_prod.predict(np.array([[201912]]))[0]
    predicciones[prod_id] = pred_201912

# Convertimos el diccionario a DataFrame para visualizar
df_predicciones = pd.DataFrame(list(predicciones.items()), columns=['product_id', 'pred_201912'])
df_predicciones.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calcula el accuracy de las predicciones usando el error absoluto medio (MAE) y el error cuadrático medio (RMSE)
# Solo para productos que tienen valor real en 201912

# Extraer los valores reales de 201912 para cada producto
valores_reales = df[df['periodo'] == 201912][['product_id', 'tn']]

# Unir con las predicciones
df_eval = pd.merge(df_predicciones, valores_reales, on='product_id', how='inner')

# Calcular métricas

mae = mean_absolute_error(df_eval['tn'], df_eval['pred_201912'])
rmse = mean_squared_error(df_eval['tn'], df_eval['pred_201912'], squared=False)

print(f"MAE (Error absoluto medio): {mae:.4f}")
print(f"RMSE (Raíz del error cuadrático medio): {rmse:.4f}")

In [ ]:
df_eval.head(20)

In [ ]:
mape = np.mean(np.abs((df_eval['tn'] - df_eval['pred_201912']) / df_eval['tn'])) * 100
print(f"MAPE (Error porcentual absoluto medio): {mape:.2f}%")

In [ ]:
# Desvío promedio porcentual de las predicciones respecto a los valores reales
desvio_promedio = np.mean(np.abs(df_eval['pred_201912'] - df_eval['tn']) / df_eval['tn']) * 100
print(f"Desvío promedio porcentual: {desvio_promedio:.2f}%")

In [ ]:


# Selecciona la serie temporal del producto 20001
serie_20001 = pivot_df[pivot_df['product_id'] == 20001].iloc[0, 1:]  # Excluye la columna product_id
serie_20001 = serie_20001.dropna()

# Convierte el índice a periodo de fecha para la serie temporal
serie_20001.index = pd.to_datetime(serie_20001.index.astype(str), format='%Y%m')

# Ajusta el modelo SARIMAX considerando estacionalidad trimestral (4 meses por estación)
model_sarimax = SARIMAX(serie_20001, order=(1,1,1), seasonal_order=(1,1,1,4))
result_sarimax = model_sarimax.fit(disp=False)

# Predicción para diciembre 2019
pred_sarimax = result_sarimax.get_prediction(start=pd.to_datetime('2019-12-01'), end=pd.to_datetime('2019-12-01'))
pred_value = pred_sarimax.predicted_mean.iloc[0]

print(f"Predicción SARIMAX para product_id=20001 en 2019-12: {pred_value:.5f}")

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarimax_preds = []
sarimax_real = []

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]
    serie = serie.dropna()
    if len(serie) < 8 or 201912 not in serie.index:
        continue  # Necesitamos suficientes datos y valor real en 201912

    # Convertir el índice a fechas para SARIMAX
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')

    # SARIMAX simple, estacionalidad trimestral (4)
    try:
        model = SARIMAX(serie.iloc[:-1], order=(1,1,1), seasonal_order=(1,1,1,4))
        result = model.fit(disp=False)
        pred = result.get_prediction(start=serie.index[-1], end=serie.index[-1])
        pred_value = pred.predicted_mean.iloc[0]
        real_value = serie.iloc[-1]
        sarimax_preds.append(pred_value)
        sarimax_real.append(real_value)
    except Exception as e:
        continue  # Si falla el ajuste, lo salteamos

sarimax_preds = np.array(sarimax_preds)
sarimax_real = np.array(sarimax_real)
sarimax_mape = np.mean(np.abs((sarimax_real - sarimax_preds) / sarimax_real)) * 100
print(f"MAPE SARIMAX (Error porcentual absoluto medio): {sarimax_mape:.2f}%")

In [ ]:
# Suma total de toneladas reales para diciembre 2019
total_real_dic2019 = valores_reales['tn'].sum()

# Suma total de toneladas predichas para diciembre 2019 (usar df_predicciones)
total_pred_dic2019 = df_predicciones['pred_201912'].sum()

print(f"Suma total real de toneladas en diciembre 2019: {total_real_dic2019:.2f}")
print(f"Suma total predicha de toneladas en diciembre 2019: {total_pred_dic2019:.2f}")

In [ ]:
# Predicción Seasonal Naive Forecast para diciembre 2019 (2019-12-01)
# Usamos el valor de diciembre 2018 (2018-12-01) como predicción para diciembre 2019

# Proyección lineal simple usando solo los valores de 2017-12-01 y 2018-12-01 para cada producto
linear_simple_preds = []
linear_simple_real = []

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]
    serie = serie.dropna()
    # Convertir el índice a fechas
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')
    # Necesitamos valores para 2017-12-01, 2018-12-01 y 2019-12-01
    if (
        pd.Timestamp('2017-12-01') in serie.index and
        pd.Timestamp('2018-12-01') in serie.index and
        pd.Timestamp('2019-12-01') in serie.index
    ):
        x = np.array([201712, 201812]).reshape(-1, 1)
        y = np.array([serie.loc[pd.Timestamp('2017-12-01')], serie.loc[pd.Timestamp('2018-12-01')]])
        model = LinearRegression()
        model.fit(x, y)
        pred = model.predict(np.array([[201912]]))[0]
        real = serie.loc[pd.Timestamp('2019-12-01')]
        linear_simple_preds.append(pred)
        linear_simple_real.append(real)

linear_simple_preds = np.array(linear_simple_preds)
linear_simple_real = np.array(linear_simple_real)
linear_simple_mape = np.mean(np.abs((linear_simple_real - linear_simple_preds) / linear_simple_real)) * 100
print(f"MAPE Linear Simple (Error porcentual absoluto medio): {linear_simple_mape:.2f}%")

In [ ]:
# La predicción para product_id=20001 en 2019-12 usando el modelo de regresión lineal ya fue calculada en la celda 16:
# pred_201912 = model.predict(np.array([[201912]]))[0]

print(linear_simple_real)

In [ ]:

# pmdarima.auto_arima: Esta biblioteca puede encontrar automáticamente los mejores órdenes (p,d,q)(P,D,Q,S) para cada serie individual,
# basándose en criterios como AIC o BIC. Esto podría mejorar la precisión por producto, pero aumentaría el tiempo de cómputo.

# Python

# Ejemplo con pmdarima para el caso actual

autoarima_preds = []
autoarima_real = []

for prod_id in pivot_df['product_id']:
    serie = pivot_df[pivot_df['product_id'] == prod_id].iloc[0, 1:]
    serie = serie.dropna()
    # Necesitamos al menos 8 datos y valor real en 201912
    if len(serie) < 8 or 201912 not in serie.index:
        continue

    # Convertir el índice a fechas
    serie.index = pd.to_datetime(serie.index.astype(str), format='%Y%m')

    # Usar todos los datos menos el último (2019-12-01) para entrenar
    train_serie = serie.iloc[:-1]
    try:
        auto_model = pm.auto_arima(
            train_serie,
            start_p=1, start_q=1,
            max_p=3, max_q=3, m=4,
            start_P=0, seasonal=True,
            d=1, D=1, trace=False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True
        )
        pred = auto_model.predict(n_periods=1)[0]
        real = serie.iloc[-1]
        autoarima_preds.append(pred)
        autoarima_real.append(real)
    except Exception as e:
        continue

autoarima_preds = np.array(autoarima_preds)
autoarima_real = np.array(autoarima_real)
autoarima_mape = np.mean(np.abs((autoarima_real - autoarima_preds) / autoarima_real)) * 100
print(f"MAPE Auto-ARIMA (Error porcentual absoluto medio): {autoarima_mape:.2f}%")

In [ ]:
# Crear un DataFrame con las predicciones y los valores reales del método seasonal naive
df_seasonal_naive = pd.DataFrame({
    'preds': autoarima_preds,
    'real': autoarima_real
})

df_seasonal_naive.head()